# Tracking token usage

This notebook goes over how to track your token usage for specific calls. It is currently only implemented for the OpenAI API and AWS Bedrock.

## OpenAI

Let's first look at an extremely simple example of tracking token usage for a single LLM call.


In [1]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI(model_name="gpt-4")

In [9]:
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(cb)

Tokens Used: 24
	Prompt Tokens: 11
	Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.0011099999999999999


Anything inside the context manager will get tracked. Here's an example of using it to track multiple calls in sequence.

In [10]:
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    result2 = llm.invoke("Tell me a joke")
    print(cb.total_tokens)

48


If a chain or agent with multiple steps in it is used, it will track all those steps.

In [11]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [12]:
with get_openai_callback() as cb:
    response = agent.run(
        "Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...

Invoking: `Search` with `Olivia Wilde's current boyfriend`


['Things are looking golden for Olivia Wilde, as the actress has jumped back into the dating pool following her split from Harry Styles — read ...', "“I did not want service to take place at the home of Olivia's current partner because Otis and Daisy might be present,” Sudeikis wrote in his ...", "February 2021: Olivia Wilde praises Harry Styles' modesty. One month after the duo made headlines with their budding romance, Wilde gave her new beau major ...", 'An insider revealed to People that the new couple had been dating for some time. "They were in Montecito, California this weekend for a wedding, ...', 'A source told People last year that Wilde and Styles were still friends despite deciding to take a break. "He\'s still touring and is now going ...', "... love life. “He's your typical average Joe.” The source adds, “She's not giving too much away right now and wants to keep the rela

## Bedrock

In [4]:
from langchain.callbacks import get_bedrock_token_count_callback
from langchain.llms import Bedrock
import anthropic

In [5]:
llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    region_name="us-east-1",
    model_kwargs={"temperature": 0.0},
)

In [6]:
with get_bedrock_token_count_callback() as cb:
    text = "\n\nHuman: What is the square root of 4?\n\nAssistant: "
    answer = llm(text)
    print(answer)  # check if invocations return same response
    answer = llm(text)
    print(answer)  # check if invocations return same response

    total_tokens = cb.total_tokens
    print(cb.total_tokens, cb.prompt_tokens, cb.completion_tokens)
    client = anthropic.Anthropic()  # check token count with Anthropic's own token counter
    assert cb.prompt_tokens == client.count_tokens(text) * 2, "Input token count does not match"
    
    # to check total token count we need to include the generation stop sequence that is not included in answers
    total_token_count = client.count_tokens(text + "The square root of 4 is 2." + "\n\nHuman: ") * 2
    assert cb.total_tokens == total_token_count

 The square root of 4 is 2.
 The square root of 4 is 2.
58 34 24
